# Data Scraping

In [3]:
# import necessary libraries
import requests
from bs4 import BeautifulSoup as bs
from time import *
from random import randint


url = "https://en.wikipedia.org/wiki/List_of_chemistry_journals"

def get_list_journals_page(url):
    
    """
    send request to a server to get a page of all journals listed
    """
    
    # specify the parameters of the browser settings
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) \
    Chrome/83.0.4103.61 Safari/537.36'}
    page = requests.get(url, headers = header)
    bsobj = bs(page.content,'lxml')
    return [page.status_code, bsobj]

In [21]:
# getting status code and the soup object
status_code = get_list_journals_page(url)[0]
print('Status code', status_code)

bsobj = get_list_journals_page(url)[1]

Status code 200


In [9]:
def links_to_journal(bsobj):
    
    """
    search soup object (bsobj) to find links to a specific journal
    """
 
    links = []
    for li in bsobj.findAll('li'):
        if not li.i:
            continue
        links.append(li.i.a['href'])
    return links

In [22]:
links = links_to_journal(bsobj)

In [15]:
def full_journal_path(links):
    
    """
    compile the full path for a journal,
    that consis from the base_path and the individual links-pointers
    """

    base_path = "https://en.wikipedia.org/"

    full_links = []
    for link in links:
        full = base_path + link
        full_links.append(full)
    return full_links

In [23]:
full_links = full_journal_path(links)

In [17]:
# sample scraping, for full scraping remove list slice
full_links = full_links[:5]

# dict to store key and values
movie_info = {}

# list to append dict
get_info = []

#tracker of pages
n = 0

# traker of links
tracker = list()


def dataframe_creation(movie_info, get_info,tracker, n, full_links):
    
    """
    iterate across full links and add it to a tracker 
    to avoid scraping the same link again
    """
    
    for link in full_links:
        if link in tracker:
            continue
        tracker.append(link)

        """
        to avoid being blocked due to sending excessive server requests
        """
        x = randint(10,17)
        sleep(x)
        print('time is', x, 'scraping page', n)

        r = requests.get(link)
        soup = bs(r.content)

        """
        for individual links find table to scrape in soup object
        and create a dictionary where keys would be the th and values are td
        """
        
        info_box = soup.find(class_="infobox hproduct")
        try:
            info_rows = info_box.find_all("tr")
        except:
            continue

        for index, row in enumerate(info_rows):
            if not row.find('th'):
                continue
            content_key = row.find("th").get_text(" ", strip=True)
            try:
                content_value = row.find("td").get_text()
            except:
                pass
            movie_info[content_key] = content_value

        """
        as title not in that class ("infobox hproduct") search the entire soup object 
        to find journal title and its description
        """
        
        for k in info_box.find_all('span', {'class':"fn"}):
            res = k.text
        movie_info['title'] = res

        try:
            p = soup.find(class_="mw-parser-output")
            parag = p.p.get_text()
            movie_info['description'] = parag
        except:
            pass
        get_info.append(movie_info)
        movie_info = {}
        n+=1

    df = pd.DataFrame(get_info)

    return df

In [18]:
df = dataframe_creation(movie_info, get_info,tracker, n, full_links)

time is 15 scraping page 0
time is 12 scraping page 1
time is 16 scraping page 2
time is 16 scraping page 3
time is 13 scraping page 4


<IPython.core.display.Javascript object>

In [20]:
# get the datasset head
df.head()

,Discipline,Language,Edited by,Publication details,History,Publisher,Frequency,Impact factor,Standard abbreviations ISO 4 ( alt ) · Bluebook ( alt1 · alt2 ) NLM ( alt ) · MathSciNet ( alt ),ISO 4,...,CODEN,ISSN,LCCN,OCLC no.,Links,title,description,Standard abbreviations ISO 4 ( alt ) · Bluebook ( alt1 · alt2 ) NLM · MathSciNet,Former name(s),Open access
0,"Chemistry, biochemistry",English,Cynthia J. Burrows,Cynthia J. Burrows,1968–present,American Chemical Society (United States),Monthly,22.384 (2020),22.384 (2020),Acc. Chem. Res.,...,ACHRE4,0001-4842 (print)1520-4898 (web),70008300,475052789,475052789,Accounts of Chemical Research,Accounts of Chemical Research is a monthly pee...,NaN,NaN,NaN
1,Chemistry,"English, German, French",NaN,"English, German, French",1947-1999,"Danish Chemical Society, Finnish Chemical Soci...",NaN,NaN,NaN,Acta Chem. Scand.,...,NaN,NaN,NaN,NaN,Acta Chem. Scand.,Acta Chemica Scandinavica,Acta Chemica Scandinavica was a peer-reviewed ...,"Danish Chemical Society, Finnish Chemical Soci...",NaN,NaN
2,Chemistry,"English, Slovene",Aleksander Pavko,Aleksander Pavko,1954–present,Slovenian Chemical Society (Slovenia),Quarterly,1.104 (2017),1.104 (2017),Acta Chim. Slov.,...,ACSLE7,1318-0207 (print)1580-3155 (web),94646636,29881395,29881395,Acta Chimica Slovenica,Acta Chimica Slovenica is a quarterly scientif...,NaN,Vestnik Slovenskega kemijskega društva,NaN
3,Materials Science,English,Joern Ritterbusch,Joern Ritterbusch,1985–present,Wiley-VCH (Germany),weekly,18.808 (2020),18.808 (2020),Adv. Funct. Mater.,...,AFMDC6,1616-301X (print)1616-3028 (web),2001205770,NaN,2001205770,Advanced Functional Materials,Advanced Functional Materials is a peer-review...,NaN,Advanced Materials for Optics and Electronics ...,some
4,Chemistry,English,NaN,English,1968–present,Sigma-Aldrich (USA),NaN,17.083[1] (2014),17.083[1] (2014),Aldrichimica Acta,...,ALACBI,0002-5100,NaN,NaN,0002-5100,Aldrichimica Acta,Aldrichimica Acta is a scientific journal publ...,NaN,NaN,NaN


In [108]:
# save dataset for future use
df.to_csv('chem_journ.csv')

# Data Cleaning

In [20]:
# import libraries
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# import dataframe
df = pd.read_csv(r'C:\Users\valer\Desktop\Projects\Chem\chem_journ.csv')



In [21]:
df.head()

,Unnamed: 0,Discipline,Language,Edited by,Publication details,History,Publisher,Frequency,Impact factor,Standard abbreviations ISO 4 ( alt ) · Bluebook ( alt1 · alt2 ) NLM ( alt ) · MathSciNet ( alt ),ISO 4,Indexing CODEN · JSTOR ( alt ) · LCCN ( alt ) MIAR · NLM ( alt ) · Scopus,CODEN,ISSN,LCCN,OCLC no.,Links,title,description,Standard abbreviations ISO 4 ( alt ) · Bluebook ( alt1 · alt2 ) NLM · MathSciNet,Former name(s),Open access,NLM,License,Editor,Categories,Year founded,Country,Website,OCLC,Indexing CODEN · JSTOR · LCCN MIAR · NLM · Scopus
0,0,"Chemistry, biochemistry",English,Cynthia J. Burrows,Cynthia J. Burrows,1968–present,American Chemical Society (United States),Monthly,22.384 (2020),22.384 (2020),Acc. Chem. Res.,Acc. Chem. Res.,ACHRE4,0001-4842 (print)1520-4898 (web),70008300,475052789.0,475052789,Accounts of Chemical Research,Accounts of Chemical Research is a monthly pee...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Chemistry,"English, German, French",NaN,"English, German, French",1947-1999,"Danish Chemical Society, Finnish Chemical Soci...",NaN,NaN,NaN,Acta Chem. Scand.,NaN,NaN,NaN,NaN,NaN,Acta Chem. Scand.,Acta Chemica Scandinavica,Acta Chemica Scandinavica was a peer-reviewed ...,"Danish Chemical Society, Finnish Chemical Soci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Chemistry,"English, Slovene",Aleksander Pavko,Aleksander Pavko,1954–present,Slovenian Chemical Society (Slovenia),Quarterly,1.104 (2017),1.104 (2017),Acta Chim. Slov.,Acta Chim. Slov.,ACSLE7,1318-0207 (print)1580-3155 (web),94646636,29881395.0,29881395,Acta Chimica Slovenica,Acta Chimica Slovenica is a quarterly scientif...,NaN,Vestnik Slovenskega kemijskega društva,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Materials Science,English,Joern Ritterbusch,Joern Ritterbusch,1985–present,Wiley-VCH (Germany),weekly,18.808 (2020),18.808 (2020),Adv. Funct. Mater.,Adv. Funct. Mater.,AFMDC6,1616-301X (print)1616-3028 (web),2001205770,NaN,2001205770,Advanced Functional Materials,Advanced Functional Materials is a peer-review...,NaN,Advanced Materials for Optics and Electronics ...,some,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Chemistry,English,NaN,English,1968–present,Sigma-Aldrich (USA),NaN,17.083[1] (2014),17.083[1] (2014),Aldrichimica Acta,Aldrichimica Acta,ALACBI,0002-5100,NaN,NaN,0002-5100,Aldrichimica Acta,Aldrichimica Acta is a scientific journal publ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# get only those columns that are important for a project
df = df[['Discipline', 'Language', 'Edited by', 'History',
    'Frequency', 'Impact factor', 'description',
    'title']]

In [23]:
def data_preprocessing(df):
    
    """
    extract the imact factor number from a given column, convert its value to float and drop the initial column 
    """
    df['Impact_factor'] = df['Impact factor'].str.replace('(','').str.replace(')','').str.replace(r'\d{4,7}','').str.replace('\[\d+]', '')
    df['Impact_factor'] = pd.to_numeric(df['Impact_factor'], errors = 'coerce')
    df['Impact_factor'] = df['Impact_factor'].astype(float)
    df.drop('Impact factor', axis = 1, inplace = True)
    
    df['description'] = df['description'].str.lower()
    
    
    return df

In [24]:
df = data_preprocessing(df)

In [25]:
def get_index(key_words):
    
    """
    getting index of the journal if the key words match the journal description info
    """
    res = key_words.lower().split()
    lst_index = []
    
    for word in res:
        for row in range(df.shape[0]):
            if df['description'].str.contains(word)[row]:
                lst_index.append(row)
    return  lst_index


def dic_journals(lst_index):
    
    """
    get the title and the impact factor info based om the index match from get_index function
    """
    lst_journals = []
    lst_factor = []
    dic = {}

    for row in lst_index:

        journal_name = df['title'][row]
        impact_factor = df['Impact_factor'][row]
        lst_factor.append(impact_factor)
        lst_journals.append(journal_name)

        dic = dict(zip(lst_journals, lst_factor))
    return dic

def create_dataframe(dic):
    
    """
    create a dataset based on the dict and sort values by impact factor
    """   
    df_new = pd.DataFrame(dic.values(), dic.keys()).reset_index()
    df_new.columns = ['title', 'Impact_factor']
    df_new = df_new.sort_values(by = 'Impact_factor', ascending = False)
    return df_new

# Model Building

In [31]:
import pandas as pd
df = pd.read_csv(r'C:\Users\valer\Desktop\Projects\Chem\chem_journ.csv')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [32]:
df = df[['Discipline', 'Language', 'Edited by', 'History',
    'Frequency', 'Impact factor', 'description',
    'title']]

df = df[['title', 'Impact factor','description']]


df['Impact_factor'] = df['Impact factor'].str.replace('(','').str.replace(')','').str.replace(r'\d{4,7}','').str.replace('\[\d+]', '')
df['Impact_factor'] = pd.to_numeric(df['Impact_factor'], errors = 'coerce')
df['Impact_factor'] = df['Impact_factor'].astype(float)
df.drop('Impact factor', axis = 1, inplace = True)

In [33]:
df['description'] = df['description'].str.lower()

In [35]:
df

,title,description,Impact_factor
0,Accounts of Chemical Research,accounts of chemical research is a monthly pee...,22.384
1,Acta Chemica Scandinavica,acta chemica scandinavica was a peer-reviewed ...,NaN
2,Acta Chimica Slovenica,acta chimica slovenica is a quarterly scientif...,1.104
3,Advanced Functional Materials,advanced functional materials is a peer-review...,18.808
4,Aldrichimica Acta,aldrichimica acta is a scientific journal publ...,17.083
5,Analyst,analyst is a biweekly peer-reviewed scientific...,3.864
6,Analytica Chimica Acta,analytica chimica acta is a peer-reviewed scie...,5.977
7,Analytical and Bioanalytical Chemistry,analytical and bioanalytical chemistry is a pe...,3.286
8,Analytical Chemistry,analytical chemistry is a biweekly peer-review...,6.785
9,Angewandte Chemie,angewandte chemie (german pronunciation: [ˈaŋɡ...,15.336


In [42]:
list(df['Impact_factor'])

[22.384,
 nan,
 1.104,
 18.808,
 17.083,
 3.864,
 5.977,
 3.286,
 6.785,
 15.335999999999999,
 nan,
 nan,
 nan,
 10.745,
 23.643,
 12.703,
 3.5810000000000004,
 2.915,
 1.253,
 1.226,
 2.622,
 4.0969999999999995,
 4.349,
 5.667000000000001,
 1.723,
 3.641,
 2.42,
 4.488,
 1.171,
 5.726,
 1.216,
 3.1639999999999997,
 5.996,
 2.029,
 60.622,
 9.346,
 54.56399999999999,
 nan,
 1.9409999999999998,
 5.236000000000001,
 1.55,
 9.811,
 2.109,
 3.1239999999999997,
 2.303,
 3.1439999999999997,
 2.753,
 0.5379999999999999,
 2.753,
 3.3819999999999997,
 4.052,
 nan,
 38.532,
 3.4210000000000003,
 2.923,
 2.529,
 6.51,
 2.889,
 3.427,
 1.757,
 4.256,
 9.405,
 2.309,
 5.165,
 5.816,
 2.263,
 nan,
 3.466,
 nan,
 5.279,
 15.419,
 3.125,
 5.157,
 2.5380000000000003,
 1.444,
 6.921,
 1.758,
 4.5489999999999995,
 3.48,
 1.406,
 nan,
 nan,
 2.1959999999999997,
 5.513999999999999,
 1.5,
 4.169,
 1.3019999999999998,
 3.3810000000000002,
 3.3760000000000003,
 3.012,
 2.6,
 0.787,
 2.381,
 6.626,
 7.44600000

In [40]:
title = ['Accounts of Chemical Research',
 'Acta Chemica Scandinavica',
 'Acta Chimica Slovenica',
 'Advanced Functional Materials',
 'Aldrichimica Acta',
 'Analyst',
 'Analytica Chimica Acta',
 'Analytical and Bioanalytical Chemistry',
 'Analytical Chemistry',
 'Angewandte Chemie',
 'Annual Reports on the Progress of Chemistry',
 'Annual Reports on the Progress of Chemistry',
 'Annual Reports on the Progress of Chemistry',
 'Annual Review of Analytical Chemistry',
 'Annual Review of Biochemistry',
 'Annual Review of Physical Chemistry',
 'Applied Organometallic Chemistry',
 'Applied Spectroscopy Reviews',
 'Arkivoc',
 'Australian Journal of Chemistry',
 'Beilstein Journal of Organic Chemistry',
 'Biochemical Journal',
 'Bioconjugate Chemistry',
 'Biomacromolecules',
 'Biomedical Chromatography',
 'Bioorganic & Medicinal Chemistry',
 'Bioorganic & Medicinal Chemistry Letters',
 'Bulletin of the Chemical Society of Japan',
 'Canadian Journal of ChemistryRevue canadienne de chimie',
 'Catalysis Science & Technology',
 'Open Chemistry',
 'ChemBioChem',
 'ChemComm',
 'Chemical Physics Letters',
 'Chemical Reviews',
 'Chemical Science',
 'Chemical Society Reviews',
 'Berichte der Deutschen Chemischen Gesellschaft',
 'Chemistry Education Research and Practice',
 'Chemistry—A European Journal',
 'Chemistry Letters',
 'Chemistry of Materials',
 'ChemistrySelect',
 'ChemMedChem',
 'Chemometrics and Intelligent Laboratory Systems',
 'ChemPhysChem',
 'ChemPlusChem',
 'Chimica Oggi – Chemistry Today',
 'ChemPlusChem',
 'CrystEngComm',
 'Dalton Transactions',
 'Dalton Transactions',
 'Energy & Environmental Science',
 'Energy & Fuels',
 'Environmental Chemistry',
 'European Journal of Inorganic Chemistry',
 'European Journal of Medicinal Chemistry',
 'European Journal of Organic Chemistry',
 'Faraday Discussions',
 'Journal of the Chemical Society, Faraday Transactions',
 'Geostandards and Geoanalytical Research',
 'Green Chemistry',
 'Helvetica Chimica Acta',
 'Inorganic Chemistry',
 'International Journal of Hydrogen Energy',
 'International Journal of Quantum Chemistry',
 'Ion Exchange Letters',
 'Journal of Analytical Atomic Spectrometry',
 'Jahres-Bericht über die Leistungen der chemischen Technologie',
 'Journal of Agricultural and Food Chemistry',
 'Journal of the American Chemical Society',
 'Journal of Applied Polymer Science',
 'Journal of Biological Chemistry',
 'Journal of Biological Inorganic Chemistry',
 'Journal of the Brazilian Chemical Society',
 'Journal of Catalysis',
 'Journal of Chemical Education',
 'Journal of Chemical Information and Modeling',
 'The Journal of Chemical Physics',
 'Journal of Chemical Sciences',
 'Journal of the Chemical Society',
 'Journal of the Chemical Society of Pakistan',
 'The Journal of Chemical Thermodynamics',
 'Journal of Cheminformatics',
 'Journal of Chemometrics',
 'Journal of Chromatography A',
 'Journal of Cluster Science',
 'ACS Combinatorial Science',
 'Journal of Computational Chemistry',
 'Journal of Electroanalytical Chemistry',
 'Environmental Science: Processes & Impacts',
 'Journal of Heterocyclic Chemistry',
 'Journal of Mass Spectrometry',
 'Journal of Materials Chemistry',
 'Journal of Medicinal Chemistry',
 'Journal of Molecular Structure',
 'Computational and Theoretical Chemistry',
 'Journal of Natural Products',
 'The Journal of Organic Chemistry',
 'Journal of Organometallic Chemistry',
 'The Journal of Physical Chemistry A',
 'The Journal of Physical Chemistry B',
 'The Journal of Physical Chemistry C',
 'The Journal of Physical Chemistry Letters',
 'Journal of Polymer Science',
 'Journal of Polymer Science',
 'Journal of Radioanalytical and Nuclear Chemistry',
 'Journal of the Royal Institute of Chemistry',
 'Journal of the Electrochemical Society',
 'Journal of Thermal Analysis and Calorimetry',
 'Lab on a Chip',
 'Langmuir',
 'Liebigs Annalen',
 'Macromolecules',
 'Magnetic Resonance in Chemistry',
 'Microchimica Acta',
 'Molbank',
 'Molecular Omics',
 'Molecular Physics',
 'Molecules',
 'Nano Letters',
 'Natural Product Reports',
 'Nature Chemical Biology',
 'Nature Chemistry',
 'Nature Materials',
 'Nature Protocols',
 'New Journal of Chemistry',
 'Open Chemistry',
 'Organic & Biomolecular Chemistry',
 'Organic Letters',
 'Organometallics',
 'Perkin Transactions',
 'Photochemical & Photobiological Sciences',
 'Physical Chemistry Chemical Physics',
 'Polish Journal of Chemistry',
 'Polyhedron',
 'Proceedings of the Chemical Society',
 'RSC Advances',
 'Revista Boliviana de Química',
 'Revista de la Sociedad Venezolana Química',
 'Scientia Pharmaceutica',
 'Soft Matter',
 'Spectroscopy Letters',
 'Surface Science Reports',
 'Synlett',
 'Synthesis',
 'Science China',
 'Talanta',
 'Tetrahedron',
 'Tetrahedron Letters',
 'Theoretical Chemistry Accounts',
 'Trends in Analytical Chemistry',
 'Zeitschrift für Naturforschung',
 'Zeitschrift für Naturforschung B',
 'Zeitschrift für Physikalische Chemie']

In [41]:
description = ['accounts of chemical research is a monthly peer-reviewed scientific journal published by the american chemical society containing overviews of basic research and applications in chemistry and biochemistry. it was established in 1968 and the editor-in-chief is cynthia j. burrows (university of utah).[1]\n',
 'acta chemica scandinavica was a peer-reviewed nordic scientific journal in the fields of chemistry. the journal was established in 1947 and was jointly managed during 53 years from beginning of 1947 until end of 1999 by the chemical societies in denmark (the danish chemical society), finland (finnish chemical societies), norway (the norwegian chemical society) and sweden (the swedish chemical society). the journal was jointly owned by the four chemical societies through the publishing association acta chemica scandinavica. the swedish chemical society managed the administration of the publishing association at their secretariat in stockholm.\n',
 'acta chimica slovenica is a quarterly scientific journal of chemistry. it comprises two parts: the first part contains peer-reviewed scientific and expert articles from the various fields of chemistry, written in english and accompanied by abstracts in slovene. the second part, written in slovene, contains societal news: lists of newly conferred academic degrees, reports on the work of the sections of the slovenian chemical society, expert articles and book reviews, and news on conferences and other meetings. the journal and the articles published since 1998 are also available online. the journal was established in 1954 as vestnik slovenskega kemijskega društva and obtained its current name in 1993.\n',
 'advanced functional materials is a peer-reviewed scientific journal, published by wiley-vch. established in february 2001, the journal began to publish monthly in 2002 and moved to 18/year in 2006,  biweekly in 2008, and weekly in 2013.\n',
 'aldrichimica acta is a scientific journal published by sigma-aldrich. established in 1968 in milwaukee, wi, aldrichimica acta publishes reviews in the field of synthetic organic chemistry, with each issue focusing on a special topic. the journal is open access. in 2015, the acta was ranked #1 among journals in the field of organic chemistry by impact factor.[2]\n',
 'analyst is a biweekly peer-reviewed scientific journal covering all aspects of analytical chemistry, bioanalysis, and detection science. it is published by the royal society of chemistry and the editor-in-chief is duncan graham (university of strathclyde). the journal was established in 1876 by the society for analytical chemistry.\n',
 'analytica chimica acta is a peer-reviewed scientific journal published since 1947 that covers original research and reviews of fundamental and applied aspects of analytical chemistry.[1]\n',
 'analytical and bioanalytical chemistry is a peer-reviewed scientific journal publishing research articles in the broad field of analytical and bioanalytical chemistry.[1] some of the subjects covered are the development of instruments for mass spectrometry, metallomics, ionics, and the analytical characterization of nano- and biomaterials. the journal was first published in 1862 under the name fresenius’ journal of analytical chemistry. in 2002 the journal was renamed to analytical and bioanalytical chemistry.\n',
 'analytical chemistry is a biweekly peer-reviewed scientific journal published since 1929 by the american chemical society. articles address general principles of chemical measurement science and novel analytical methodologies. topics commonly include chemical reactions and selectivity, chemometrics and data processing, electrochemistry, elemental and molecular characterization, imaging, instrumentation, mass spectrometry, microscale and nanoscale systems, -omics, sensing, separations, spectroscopy, and surface analysis. it is abstracted and indexed in chemical abstracts service, cab international, ebscohost, proquest, pubmed, scopus, and the science citation index expanded. according to the journal citation reports, it has a 2019 impact factor of 6.785[1] and is the top analytical chemistry journal ranked by h-index in the scimago journal rank.[2] the editor-in-chief is jonathan v. sweedler (university of illinois).\n',
 'angewandte chemie (german pronunciation: [ˈaŋɡəˌvantə çeˈmiː], meaning "applied chemistry") is a weekly peer-reviewed scientific journal that is published by wiley-vch on behalf of the german chemical society (gesellschaft deutscher chemiker). publishing formats include feature-length reviews, short highlights, research communications, minireviews, essays, book reviews, meeting reviews, correspondences, corrections, and obituaries. this journal contains review articles covering all aspects of chemistry. in 2018[update] its impact factor was 12.257, and it was ranked 17th out of 172 in the subject category "chemistry, multidisciplinary."[1]\n',
 'annual reports on the progress of chemistry was a yearly review journal published by the royal institute of chemistry and after 1980 the royal society of chemistry. it was established in 1904. in 1967 the journal was split into two sections, a and b, covering inorganic and organic chemistry, respectively. in 1980, a third series was started, c, covering physical chemistry. the journal was discontinued in 2013.\n',
 'annual reports on the progress of chemistry was a yearly review journal published by the royal institute of chemistry and after 1980 the royal society of chemistry. it was established in 1904. in 1967 the journal was split into two sections, a and b, covering inorganic and organic chemistry, respectively. in 1980, a third series was started, c, covering physical chemistry. the journal was discontinued in 2013.\n',
 'annual reports on the progress of chemistry was a yearly review journal published by the royal institute of chemistry and after 1980 the royal society of chemistry. it was established in 1904. in 1967 the journal was split into two sections, a and b, covering inorganic and organic chemistry, respectively. in 1980, a third series was started, c, covering physical chemistry. the journal was discontinued in 2013.\n',
 'the annual review of analytical chemistry is a peer-reviewed academic journal that publishes an annual volume of review articles relevant to analytical chemistry. it was established in 2008 and is published by annual reviews.\n',
 'annual review of biochemistry is an annual peer reviewed scientific journal published by annual reviews, a nonprofit scientific publisher. its first volume was published in 1932, and its founding editor was j. murray luck. the current editor is roger d. kornberg. the journal focuses on molecular biology and biological chemistry review articles. as of 2021, journal citation reports gives the journal an impact factor of 23.643, ranking it fourth out of 298 journals in the category "biochemistry and molecular biology".[1]\n',
 'annual review of physical chemistry is a peer-reviewed scientific journal published by annual reviews. it covers all topics pertaining to physical chemistry. the editors are mark a. johnson (yale university) and todd j. martínez (stanford university).[2] the journal is indexed in the science citation index expanded and chemical abstracts service.[3] as of 2021, journal citation reports gives it a 2020 impact factor of 12.703.[1]\n',
 'applied organometallic chemistry is a monthly peer-reviewed scientific journal published since 1987 by john wiley & sons. \nthe editor-in-chief is cornelis j. elsevier (university of amsterdam).\n',
 'applied spectroscopy reviews is a peer reviewed scientific journal that publishes review articles on all aspects of spectroscopy.\n',
 'arkivoc (archive for organic chemistry) is a peer-reviewed open access scientific journal covering all aspects of organic chemistry. it is published by the non-profit organization arkat usa, which was established in 2000 through a personal donation from alan r. katritzky and linde katritzky. arkivoc is the primary publication of arkat usa. according to the journal citation reports, the journal has a 2014 impact factor of 1.165, ranking it 37th out of 57 journals in the category "chemistry, organic".[1]\n',
 'the australian journal of chemistry - an international journal for chemical science is a monthly peer-reviewed scientific journal published by csiro publishing. it was established in 1948 and covers all aspects of chemistry. the editors-in-chief are george koutsantonis (university of western australia) and john wade (university of melbourne).\n',
 'the beilstein journal of organic chemistry is a peer-reviewed open-access scientific journal established in 2005. it is published by the beilstein institute for the advancement of chemical sciences, a german non-profit foundation. the editor-in-chief is peter seeberger (max planck institute of colloids and interfaces).\n',
 'the biochemical journal is a peer-reviewed scientific journal which covers all aspects of biochemistry, as well as cell and molecular biology. it is published by portland press and was established in 1906.\n',
 'bioconjugate chemistry is a peer-reviewed scientific journal on bioconjugation, published since 1990 by the american chemical society. bioconjugate chemistry is currently indexed in the following archives: chemical abstracts service (cas), scopus, ebscohost, proquest, british library, pubmed, and web of science.\n',
 'biomacromolecules is a peer-reviewed scientific journal published since 2000 by the american chemical society. it is abstracted and indexed in chemical abstracts service, scopus, ebscohost, pubmed, and science citation index expanded. as of 2010[update], the editor in chief is ann-christine albertsson. according to the journal citation reports, the journal has a 2014 impact factor of 5.750.[1]\n',
 'biomedical chromatography is a monthly peer-reviewed scientific journal, published since 1986 by john wiley & sons. it covers research on the applications of chromatography and allied techniques in the biological and medical sciences. the editor-in-chief is michael bartlett (university of georgia).\n',
 'bioorganic & medicinal chemistry is a scientific journal focusing on the results of research on the molecular structure of biological organisms and the interaction of biological targets with chemical agents.[1] it is published by elsevier, which also publishes the related journal bioorganic & medicinal chemistry letters.\n',
 'bioorganic & medicinal chemistry letters is a scientific journal focusing on the results of research on the molecular structure of biological organisms and the interaction of biological targets with chemical agents. it is published by elsevier, which also publishes bioorganic & medicinal chemistry for longer works.\n',
 'bulletin of the chemical society of japan (日本化学会欧文誌, nihon kagakukai ōbunshi) is a scientific journal, which was founded in 1926 by the chemical society of japan. it publishes accounts, articles, and short articles in the fields of theoretical and physical chemistry, analytical and inorganic chemistry, organic and biological chemistry, and applied and materials chemistry.\n',
 'the canadian journal of chemistry (fr. revue canadienne de chimie) is a peer-reviewed scientific journal published by nrc research press. it was established in 1951 as the continuation of canadian journal of research, section b: chemical sciences. papers are loaded to the web in advance of the printed issue and are available in both pdf and html formats.\n',
 'catalysis science & technology is a peer-reviewed scientific journal that is published monthly by the royal society of chemistry.[1][2] the editor-in-chief is javier pérez-ramírez (eth zurich, switzerland).[3]\n',
 'open chemistry is a monthly peer-reviewed open access scientific journal covering all fields of chemistry. it is published by walter de gruyter. the editor-in-chief is joaquín plumet, (complutense university).\n',
 'chembiochem is a peer-reviewed scientific journal covering chemical biology, synthetic biology, and bio-nanotechnology and published by wiley-vch on behalf of chemistry europe. the journal publishes communications, full papers, reviews, minireviews, highlights, concepts, book reviews, and conference reports. viewpoints, correspondence, essays, web sites, and databases are also occasionally featured. it is abstracted and indexed in major databases and has been online-only since 2016.[1]\n',
 'chemcomm (or chemical communications), formerly known as journal of the chemical society d: chemical communications (1969–1971), journal of the chemical society, chemical communications (1972–1995),[1] is a peer-reviewed scientific journal published by the royal society of chemistry.[2] it covers all aspects of chemistry. in january 2012, the journal moved to publishing 100 issues per year.[2] the current chair of the editorial board is douglas stephan (university of toronto, canada), while the executive editor is richard kelly.\n',
 'chemical physics letters is a biweekly peer-reviewed scientific journal covering research in chemical physics and physical chemistry. it was established in 1967 and is published by elsevier. the editors-in-chief are david c. clary, b. dietzek, k-l. han, and a. karton.\n',
 'chemical reviews is peer-reviewed scientific journal published twice per month by the american chemical society. it publishes review articles on all aspects of chemistry. it was established in 1924 by william albert noyes (university of illinois). as of 1\xa0january\xa02015[update] the editor-in-chief is sharon hammes-schiffer.[1]\n',
 'chemical science is a weekly peer-reviewed scientific journal covering all aspects of chemistry. it is the flagship journal of the royal society of chemistry.[1] it was established in july 2010 and is published by the royal society of chemistry; before 2018, it was published monthly. it won the best new journal 2011 award from the association of learned and professional society publishers.[2] the editor-in-chief is andrew ian cooper (university of liverpool).\n',
 'chemical society reviews is a biweekly peer-reviewed scientific journal published by the royal society of chemistry, for review articles on topics of current interest in chemistry. its predecessors were quarterly reviews, chemical society (1947–1971) and royal institute of chemistry, reviews (1968–1971); it maintained its current title since 1971. according to the journal citation reports, the journal has a 2020 impact factor of 54.564.[1] the current editor-in-chief (chair of editorial board) is douglas stephan.\n',
 'chemische berichte (usually abbreviated as ber. or chem. ber.) was a german-language scientific journal of all disciplines of chemistry founded in 1868. it was one of the oldest scientific  journals in chemistry, until it merged with recueil des travaux chimiques des pays-bas to form chemische berichte/recueil in 1997. chemische berichte/recueil was then merged with other european journals in 1998 to form european journal of inorganic chemistry.\n',
 'chemistry education research and practice is a quarterly peer-reviewed open access academic journal published by the royal society of chemistry covering chemistry education. the editor-in-chief is keith s. taber of the university of cambridge. the associate editors are ajda kahveci of depaul university, scott e. lewis of the university of south florida, and michael k. seery of the university of edinburgh. according to the journal citation reports, the journal has a 2015 impact factor of 1.802.[1]\n',
 'chemistry—a european journal is a weekly peer-reviewed scientific journal that covers all areas of chemistry and related fields. it is published by wiley-vch on behalf of chemistry europe. the editor-in-chief is haymo ross.\n',
 'chemistry letters is a peer-reviewed scientific journal published by the chemical society of japan. it specializes in the rapid publication of reviews and letters on all areas of chemistry. the editor-in-chief is mitsuhiko shionoya (university of tokyo). according to the journal citation reports, the journal has a 2014 impact factor of 1.23.[1]\n',
 'chemistry of materials is a peer-reviewed scientific journal, published since 1989 by the american chemical society. chemistry of materials is currently indexed in: chemical abstracts service (cas), scopus, ebscohost, british library, swetswise, and web of science.  it was founded by leonard v. interrante, who was the editor-in-chief until 2013. jillian m. buriak took over as editor-in-chief in january 2014.editor profile - chemistry of materials - acs publications. she was followed by sara e. skrabalak, who assumed the position of editor-in-chief in november 2020. \n',
 'chemistryselect  is a peer-reviewed scientific journal covering all areas of chemistry and adjacent fields. it is published by wiley-vch on behalf of chemistry europe. the journal is indexed in the science citation index expanded and scopus.[1]\n',
 'chemmedchem is a biweekly peer-reviewed scientific journal covering medicinal chemistry. it is published by wiley-vch on behalf of chemistry europe. in addition to original research in the form of full papers and shorter communications, the journal contains review-type articles (reviews, minireviews, patent reviews, essays, highlights) as well as occasional book reviews and conference reports.\n',
 'chemometrics and intelligent laboratory systems is a peer-reviewed scientific journal sponsored by the chemometrics society and published since 1986 by elsevier. the current editor-in-chief is r. tauler (barcelona, spain).\n',
 'chemphyschem is a biweekly peer-reviewed scientific journal published by wiley-vch on behalf of chemistry europe. it was established in 2000 and covers all aspects of chemical physics and physical chemistry. initially published monthly, the journal moved to 18 issues per year in 2007, and further to biweekly in 2016.\n',
 'chempluschem is a monthly peer-reviewed scientific journal covering chemistry and published by wiley-vch on behalf of chemistry europe. it was established in 1929 by e. votoček and j. heyrovský and renamed in 1939 to collection tschechischer chemischer forschungsarbeiten/collection des travaux chimiques tchèques/collection of czech chemical communications for one year. publication was suspended until 1947, when it resumed publication as collection of czechoslovak chemical communications. it obtained its current name in 2012.\n',
 'chimica oggi – chemistry today is a bimonthly peer-reviewed scientific journal published by tekno scienze publisher, covering chemistry. it was established in 1983 and the editor-in-chief is silvana maini.\n',
 'chempluschem is a monthly peer-reviewed scientific journal covering chemistry and published by wiley-vch on behalf of chemistry europe. it was established in 1929 by e. votoček and j. heyrovský and renamed in 1939 to collection tschechischer chemischer forschungsarbeiten/collection des travaux chimiques tchèques/collection of czech chemical communications for one year. publication was suspended until 1947, when it resumed publication as collection of czechoslovak chemical communications. it obtained its current name in 2012.\n',
 '\n',
 '\n',
 "education in chemistry (often referred to by its brand 'eic') is a print and online magazine covering all areas of chemistry education,[1] mainly concentrating on the teaching of chemistry in secondary schools and universities.[2] it is published by the royal society of chemistry,[3] which also publishes chemistry education research and practice, a peer-reviewed academic journal on the same topic.[1]\n",
 'energy & environmental science is a monthly peer-reviewed scientific journal publishing original (primary) research and review articles. the journal covers work of an interdisciplinary nature in the biochemical and biophysical sciences and chemical and mechanical engineering disciplines. it covers energy area.[1] energy & environmental science is published by the royal society of chemistry and the editor-in-chief is joseph hupp, (northwestern university, usa).\n',
 'energy & fuels is a peer-reviewed scientific journal published by the american chemical society. it was established in 1987. its publication frequency switched from bimonthly to monthly in 2009. the editor-in-chief is hongwei wu (curtin university).\n',
 'environmental chemistry is a peer-reviewed scientific journal published by csiro publishing. it covers all aspects of environmental chemistry, including atmospheric chemistry, (bio)geochemistry, climate change, marine chemistry, water chemistry, polar chemistry, fire chemistry, astrochemistry, earth and geochemistry, soil and sediment chemistry, and chemical toxicology. the editor-in-chief is jamie lead (university of south carolina).\n',
 'the european journal of inorganic chemistry is a weekly peer-reviewed scientific journal covering inorganic, organometallic, bioinorganic, and solid-state chemistry.[1] it is published by wiley-vch on behalf of chemistry europe.\n',
 'the european journal of medicinal chemistry is a monthly peer-reviewed scientific journal covering medicinal chemistry and published by elsevier. it was established in 1966 as chimica therapeutica (coden: chtpba) and obtained its current title in 1974. from 1974 to 1981 the journal was still subtitled as chimica therapeutica and from 1982 to 1986 the subtitle was chimie thérapeutique, indicating its french origin. and now it is the journal of the french société de chimie thérapeutique.\n',
 'the european journal of organic chemistry is a weekly peer-reviewed scientific journal covering organic chemistry. it is published by wiley-vch on behalf of chemistry europe.\n',
 'faraday discussions is a scientific journal publishing original research papers presented at a long-running series of conferences on physical chemistry, chemical physics and biophysical chemistry which are also called faraday discussions, together with a record of the comments made at the meeting. the journal was originally published by the faraday society. the journal has been published by the royal society of chemistry (rsc) since that society merged into the rsc. from 1972 to 1991, it was known as the faraday discussions of the chemical society. traditionally there have been three faraday discussions a year, however, from 2014 around eight conferences (and therefore eight volumes of the journal) are organised annually.\n',
 'the journal of the chemical society, faraday transactions was a peer-reviewed scientific journal published from 1905 until 1998. the journal was originally published by the faraday society under the name transactions of the faraday society and was renamed in 1972.\n',
 'geostandards and geoanalytical research is a quarterly peer-reviewed scientific journal covering reference materials, analytical techniques, and data quality relevant to the chemical analysis of geological and environmental samples. the journal was established in 1977 as geostandards newsletter and modified its title in 2004. the editors-in-chief are thomas c. meisel, jacinta enzweiler, mary f. horan, kathryn l. linge, christophe r. quétel and paul j. sylvester. it is published by wiley-blackwell on behalf of the international association of geoanalysts. the journal is a hybrid open-access journal, publishing both subscription and open access articles.[1]\n',
 'green chemistry is a monthly peer-reviewed scientific journal covering every aspect of sustainable chemistry and its implementation in chemical engineering. it is published by the royal society of chemistry and was established in 1999 by james clark (university of york). articles published in this journal are intended to be conceptually accessible to a wide audience. according to the journal citation reports, the journal has a 2017 impact factor of 9.405.[1]\n',
 'helvetica chimica acta is a scientific journal of chemistry established by the swiss chemical society. it is published online by john wiley & sons. the journal has a 2019 impact factor of 2.309.[1]\n',
 'inorganic chemistry is a biweekly peer-reviewed scientific journal published by the american chemical society since 1962. it covers research in all areas of inorganic chemistry.\n',
 'the international journal of hydrogen energy is a peer-reviewed scientific journal covering all aspects of  hydrogen energy, including hydrogen generation and storage. it is published by elsevier and is an official journal of the international association for hydrogen energy. established in 1976, the journal became monthly in 1982, biweekly in 2008, 36/yr in 2013, and weekly in 2015. the editor in chief is turhan nejat veziroğlu (university of miami).\n',
 'the international journal of quantum chemistry is a peer-reviewed scientific journal publishing original, primary research and review articles on all aspects of quantum chemistry, including an expanded scope focusing on aspects of materials science, biochemistry, biophysics, quantum physics, quantum information theory, etc.[1]\n',
 'ion exchange letters is an open access scientific journal. this journal was established in 2008[1] to provide a platform for students and young scientists to publish letters and short communications. journal covers following fields:\n',
 'the journal of analytical atomic spectrometry is a peer-reviewed scientific journal publishing original (primary) research and review articles covering all areas of modern spectrometry including fundamental theory, practice and analytical applications. it is published monthly by the royal society of chemistry, the editor-in-chief is may copsey. the journal replaced annual reports on analytical atomic spectroscopy (1971–1984) in 1986 and has a 2014 impact factor of 3.466.[1]\n',
 'the jahres-bericht über die leistungen der chemischen technologie was a german scientific journal on chemistry, pharmacy and metallurgy, published from 1855 to at least 1935.[1]\n',
 'the journal of agricultural and food chemistry is a weekly peer-reviewed scientific journal established in 1953 by the american chemical society.[1] since 2015, thomas hofmann (technical university of munich) has been the editor-in-chief.[2]\n',
 'the journal of the american chemical society (also known as jacs) is a weekly peer-reviewed scientific journal that was established in 1879 by the american chemical society.[2] the journal has absorbed two other publications in its history, the journal of analytical and applied chemistry (july 1893) and the american chemical journal (january 1914). it publishes original research papers in all fields of chemistry. since 2021, the journal is edited by erick m. carreira (eth zurich).[3] in 2014, the journal moved to a hybrid open access publishing model.\n',
 'the journal of applied polymer science is a peer-reviewed scientific journal covering  polymer science. the journal covers all applications of synthetic and renewably sourced polymers, including batteries and fuel cells, organic electronics, biomedical implants and drug delivery, coatings and packaging. it also covers composites, blends, elastomers, films and membranes, fibers, emulsions and latices, degradation of polymers, block co-polymers, hydrogels, foams, nanostructured polymers, as well as innovative synthesis and processing techniques. according to the journal citation reports, the journal has a 2019 impact factor of 2.52.[1]\n',
 'the journal of biological chemistry (jbc) is a weekly peer-reviewed scientific journal that was established in 1905.[1] since 1925, it is published by the american society for biochemistry and molecular biology. it covers research in areas of biochemistry and molecular biology. the editor is alex toker.[2][3] as of january 2021, the journal is fully open access.[4] in press articles are available free on its website immediately after acceptance.\n',
 'journal of biological inorganic chemistry is a peer-reviewed scientific journal. it is an official publication of the society of biological inorganic chemistry and published by springer science+business media.\n',
 'the journal of the brazilian chemical society (print issn\xa00103-5053, eissn issn\xa01678-4790, coden jocset) is a brazilian scientific journal in chemistry. it was founded in 1990 and is published by the brazilian society of chemistry (sociedade brasileira de química), located at the instituto de química da universidade de são paulo. the journal is online, and the full text is freely available. according to the journal citation reports, the journal has a 2014 impact factor of 1.129, ranking it 100th out of 157 journals in the category "chemistry multidisciplinary".[1]\n',
 'the journal of catalysis is a monthly peer-reviewed scientific journal covering research on all aspects of heterogeneous and homogeneous catalysis. it is published by elsevier and it was established in 1962 by jan hendrik de boer and p. w. selwood. the current editor-in-chief is johannes a. lercher (technische universität münchen).[1] other members of the editorial board include bert weckhuysen and joachim sauer.[1] former editors-in-chief have been f. s. stone, w. k. hall, g. l. haller, w. n. delgass, and e. iglesia.\n',
 'the journal of chemical education is a monthly peer-reviewed academic journal available in both print and electronic versions. it is published by the division of chemical education of the american chemical society and was established in 1924 by neil gordon.[1] the journal covers research on chemical education, and its target audience includes instructors of chemistry from middle school through graduate school and some scientists in commerce, industry, and government.[2]\n',
 'the journal of chemical information and modeling is a peer-reviewed scientific journal published by the american chemical society. it was established in 1961 as the journal of chemical documentation, renamed in 1975 to journal of chemical information and computer sciences, and obtained its current name in 2005. the journal covers the fields of chemical informatics and molecular modeling. the editor-in-chief is kenneth m. merz jr. (michigan state university). the journal supports open science approaches.[1]\n',
 'the journal of chemical physics is a scientific journal published by the american institute of physics that carries research papers on chemical physics.[1]  two volumes, each of 24 issues, are published annually. it was established in 1933 when journal of physical chemistry editors refused to publish theoretical works.[2]\n',
 '2015-2019: 12 issues per year\n',
 "the journal of the chemical society was a scientific journal established by the chemical society in 1849 as the quarterly journal of the chemical society. the first editor was edmund ronalds. the journal underwent several renamings, splits, and mergers throughout its history. in 1980, the chemical society merged with several other organizations into the royal society of chemistry. the journal's continuity is found in chemical communications, dalton transactions, faraday transactions, and perkin transactions, all of which are published by the royal society of chemistry.\n",
 'the journal of the chemical society of pakistan is a bimonthly peer-reviewed scientific journal covering the field of chemistry. it is published by the chemical society of pakistan and was established in 1978. the editor-in-chief is muhammad iqbal bhanger (university of karachi). the journal is abstracted and indexed in  the science citation index expanded and current contents/physical, chemical & earth sciences.[1] according to the journal citation reports, the journal did not receive and impact factor over 2012 because of "anomalous citation patterns resulting in a significant distortion of the journal impact factor".[2]\n',
 'the journal of chemical thermodynamics is a monthly peer-reviewed scientific journal covering experimental thermodynamics and thermophysics including bio-thermodynamics, calorimetry, phase equilibria, equilibrium thermodynamic properties and transport properties. it is published by elsevier. the editors-in-chief are a. pádua, j.p.m. trusler, and r. weir.\n',
 'the journal of cheminformatics is a peer-reviewed open access scientific journal that covers cheminformatics and molecular modelling.[1][2] it was established in 2009 with david wild (indiana university) and christoph steinbeck (then at embl-ebi) as founding editors-in-chief, and was originally published by chemistry central.[3] at the end of 2015, the chemistry central brand was retired and its titles, including journal of cheminformatics, were merged with the springeropen portfolio of open access journals.[4]\n',
 'the journal of chemometrics is a monthly peer-reviewed scientific journal published since 1987 by john wiley & sons. it publishes original scientific papers, reviews, and short communications on fundamental and applied aspects of chemometrics. the current editor-in-chief is paul j. gemperline (east carolina university).\n',
 'the journal of chromatography a is a peer-reviewed scientific journal publishing research papers in analytical chemistry, with a focus on techniques and methods used for the separation and identification of mixtures.\n',
 'the journal of cluster science is a quarterly peer-reviewed scientific journal covering all aspects of cluster science, including nanoclusters and nanoparticles. it is published by springer science+business media and the editor-in-chief is r.d. adams.\n',
 'acs combinatorial science (usually abbreviated as acs comb. sci.), formerly journal of combinatorial chemistry (1999-2010), was a peer-reviewed scientific journal, published since 1999 by the american chemical society.  acs combinatorial science publishes articles, reviews, perspectives, accounts and reports in the field of combinatorial chemistry. jcs is currently indexed in: chemical abstracts service (cas), scopus, ebscohost, pubmed, and web of science.\n',
 'the journal of computational chemistry is a peer-reviewed scientific journal published since 1980 by john wiley & sons. it covers research, contemporary developments in theory and methodology, and applications in all areas of computational chemistry, including ab initio quantum chemistry methods and semiempirical methods, density functional theory, molecular mechanics, molecular dynamics, statistical mechanics, cheminformatics, biomolecular structure prediction, molecular design, and bioinformatics.\n',
 'the journal of electroanalytical chemistry is a peer-reviewed scientific journal on electroanalytical chemistry, published by elsevier twice per month. it was originally established in 1959 under the current name, but was known as the journal of electroanalytical chemistry and interfacial electrochemistry  from 1967 to 1991. it is currently edited by j. m. feliu (universidad de alicante). the journal is associated with the international society of electrochemistry. while the journal is now published exclusively in english, earlier volumes sometimes published articles in french and german.\n',
 'environmental science: processes & impacts is a monthly peer-reviewed scientific journal covering all aspects of environmental science. it is published by the royal society of chemistry and kris mcneill is the editor-in-chief. the journal was established in 1999 as the journal of environmental monitoring and obtained its current title in 2013.\n',
 'journal of heterocyclic chemistry is a peer-reviewed scientific journal summarizing progress in the field of heterocycle chemistry. it is a source for the chemspider database.[citation needed]\n',
 'the journal of mass spectrometry is a peer-reviewed scientific journal covering all aspects of mass spectrometry including instrument design and development, ionization processes, mechanisms and energetics of gaseous ion reactions, spectroscopy of gaseous ions, theoretical aspects, ion structure, analysis of compounds of biological interest, methodology development, applications to elemental analysis and inorganic chemistry, computer-related applications and developments, and environmental chemistry and other fields that use innovative aspects of mass spectrometry. it was established in 1968 as organic mass spectrometry and obtained its current title in 1995.\n',
 'the journal of materials chemistry was a weekly peer-reviewed scientific journal covering the applications, properties and synthesis of new materials. it was established in 1991 and published by the royal society of chemistry. at the end of 2012 the journal was split into three independent journals: journal of materials chemistry a (energy and sustainability), journal of materials chemistry b (biology and medicine) and journal of materials chemistry c (optical, magnetic and electronic devices).[1] the editor-in-chief was liz dunn.\n',
 'the journal of medicinal chemistry is a peer-reviewed medical journal covering research in medicinal chemistry. it is published by the american chemical society. it was established in 1959 as the journal of medicinal and pharmaceutical chemistry and obtained its current name in 1963.[1] portoghese had served as editor-in-chief from 1972-2011.[1] in 2012, gunda i. georg (university of minnesota) and shaomeng wang (university of michigan) succeeded philip s. portoghese (university of minnesota) as co-editor-in -chief. in 2021, craig w. lindsley (vanderbilt university) became the new editor-in-chief, having served as the founding editor-in-chief of acs chemical neuroscience since 2010. according to the journal citation reports, the journal received its highest impact factor to date in 2020 impact factor, with an if of 7.446, ranking it 1st out of 61 journals in the category "chemistry, medicinal".[2]\n',
 'journal of molecular structure is a scientific journal published by elsevier sciencedirect since 1968.  its articles discuss molecular structure in chemistry.\n',
 'computational and theoretical chemistry is a peer-reviewed scientific journal published by elsevier. it was established in 1985 as journal of molecular structure: theochem, a spin-off of the journal of molecular structure. it obtained its current name in 2011 and covers molecular structure in theoretical chemistry.\n',
 'the journal of natural products is a monthly peer-reviewed scientific journal covering all aspects of research on the chemistry and/or biochemistry of naturally occurring compounds. it is co-published by the american society of pharmacognosy and the american chemical society. the editor-in-chief is philip j. proteau (oregon state university).\n',
 'the journal of organic chemistry,  colloquially known as joc, is a peer-reviewed[1] scientific journal for original contributions of fundamental research in all branches of theory and practice[2] in organic and bioorganic chemistry.  it is published by the publishing arm of the american chemical society, with 24 issues per year.  according to the journal citation reports, the journal had a 2017 impact factor of 4.805[3] and it is the journal that received the most cites (100,091 in 2017) in the field of organic chemistry.[2]  according to web of knowledge (and as december 2012), eleven papers from the journal have received more than 1,000 citations, with the most cited paper[4] having received 7,967 citations.  the current editor-in-chief is scott j. miller from yale university.[5]\n',
 'the journal of organometallic chemistry is a peer-reviewed scientific journal published by elsevier, covering research on organometallic chemistry. according to the journal citation reports, the journal has a 2014 impact factor of 2.173.[1]\n',
 'the journal of physical chemistry a is a scientific journal which reports  research on the chemistry of molecules - including their dynamics, spectroscopy, kinetics, structure, bonding, and quantum chemistry. it is published weekly by the american chemical society.\n',
 'the journal of physical chemistry b is a peer-reviewed scientific journal that covers research on several fields of material chemistry (macromolecules, soft matter, and surfactants) as well as statistical mechanics, thermodynamics, and biophysical chemistry.  it has been published weekly since 1997 by the american chemical society.\n',
 'the journal of physical chemistry c publishes scientific articles reporting research on several subdisciplines of physical chemistry:\n',
 'the journal of physical chemistry letters is a peer-reviewed scientific journal published by the american chemical society. the editor-in-chief is gregory d. scholes at princeton university. the journal of physical chemistry letters covers research on all aspects of physical chemistry. george c. schatz was editor-in-chief from 2010-2019. according to the journal citation reports, the journal had an impact factor of 7.329 for 2018.[1]\n',
 'journal of polymer science is a peer-reviewed journal of polymer science currently published in multiple parts by john wiley & sons. it was originally established as the journal of polymer science in 1946 by interscience publishers and the founding editor herman f. mark, but it was split in various parts in 1962. the journal has undergone re-organization several times since. in 2020, the journal will consolidate in one single publication.[1]\n',
 'journal of polymer science is a peer-reviewed journal of polymer science currently published in multiple parts by john wiley & sons. it was originally established as the journal of polymer science in 1946 by interscience publishers and the founding editor herman f. mark, but it was split in various parts in 1962. the journal has undergone re-organization several times since. in 2020, the journal will consolidate in one single publication.[1]\n',
 'the journal of radioanalytical and nuclear chemistry is a peer-reviewed scientific journal published by springer science+business media.[1]  it publishes original papers, review papers, short communications and letters on nuclear chemistry. some of the subjects covered are nuclear chemistry, radiation chemistry, nuclear power plant chemistry, radioanalytical chemistry, and environmental radiochemistry.\n',
 'the journal of the royal institute of chemistry was a scientific journal published by the royal institute of chemistry which combined with other societies in 1980 to form the royal society of chemistry (rsc). it had various names, including those with the title of the institute prior to gaining its royal charter:-\n',
 'the journal of the electrochemical society is a monthly peer-reviewed scientific journal covering the field of electrochemical science and technology. it is published by the electrochemical society. according to the journal citation reports, the journal has a 2019 impact factor of 3.721.[1]\n',
 'the journal of thermal analysis and calorimetry is a peer-reviewed scientific journal published by springer science+business media[1] in cooperation with akadémiai kiadó. formerly this journal was known as journal of thermal analysis. it publishes papers covering all aspects of calorimetry, thermal analysis, and experimental thermodynamics. some of the subjects covered are thermogravimetry, differential scanning calorimetry of all types, derivative thermogravimetry, thermal conductivity, thermomechanical analysis, and the theory and instrumentation for thermal analysis and calorimetry.\n',
 'lab on a chip is a peer-reviewed scientific journal which publishes original (primary) research and review articles on any aspect of miniaturisation at the micro and nano scale. lab on a chip is published twice monthly by the royal society of chemistry (rsc) and the editor-in-chief is abraham lee. the journal was established in 2001 and hosts other rsc publications: highlights in chemical technology and highlights in chemical biology. according to the journal citation reports, the journal has a 2016 impact factor of 6.[1]\n',
 'langmuir is a peer-reviewed scientific journal that was established in 1985 and is published by the american chemical society. it is the leading journal focusing on the science and application of systems and materials in which the interface dominates structure and function. research areas covered include surface and colloid chemistry.\n',
 "justus liebigs annalen der chemie (often cited as just liebigs annalen) was one of the oldest and historically most important journals in the field of organic chemistry worldwide. it was established in 1832 and edited by justus von liebig with friedrich wöhler until liebig's death in 1873. \n",
 'macromolecules is a peer-reviewed scientific journal that has been published since 1968 by the american chemical society. initially published bimonthly, it became monthly in 1983 and then, in 1990, biweekly.[1] macromolecules is abstracted and indexed in scopus, ebscohost, pubmed, web of science, and swetswise. the editor-in-chief is marc a. hillmyer.[2]\nits first editor was dr. field h. winslow.[3][4]\n',
 'magnetic resonance in chemistry is a monthly peer-reviewed scientific journal covering the application of nmr, esr, and nqr spectrometry in all branches of chemistry. the journal was established in 1969 and is published by john wiley & sons. the editors-in-chief are roberto r. gil (carnegie mellon university) and  gary e. martin (merck & co.).\n',
 'microchimica acta is a monthly peer-reviewed scientific journal published by springer nature. it was established in 1937 by fritz pregl. the editors-in-chief are alberto escarpa (university of alcalá) and mamas i. prodromidis (university of ioannina), who succeeded otto s. wolfbeis (university of regensburg). the journal covers research on (bio)chemical analytical methods based on the use of micro- and nanomaterials. according to the journal citation reports, the journal has a 2019 impact factor of 6.232.[1]\n',
 'molbank is a peer-reviewed, open access, scientific journal covering synthetic chemistry and natural product chemistry, established in 1997 as a section of molecules and is published by mdpi. after 2001, the journal was published on its own, and become online-only in 2009.\n',
 'molecular omics is a bimonthly peer-reviewed scientific journal published by the royal society of chemistry. it covers the interface between chemistry, the "omic" sciences, and systems biology. the editor-in-chief is robert l. moritz (institute for systems biology).\n',
 'molecular physics is a peer-reviewed scientific journal covering research on the interface between chemistry and physics, in particular chemical physics and physical chemistry. it covers both theoretical and experimental molecular science, including electronic structure, molecular dynamics, spectroscopy, reaction kinetics, statistical mechanics, condensed matter and surface science. the journal was established in 1958 and is published by taylor & francis. according to the journal citation reports, the journal has a 2017 impact factor of 1.704.[1]\n',
 'molecules is a peer-reviewed open access scientific journal that focuses on all aspects of chemistry and materials science. it was established in march 1996 and is published monthly by mdpi. from 1997 to 2001, molbank was published as a section of the journal, before splitting into its own journal. the editor-in-chief is farid chemat.\n',
 'nano letters is a monthly peer-reviewed scientific journal published by the american chemical society. it was established in january 2001. the editor-in-chief is teri w. odom (northwestern university).[1] the journal covers all aspects of nanoscience and nanotechnology and their subdisciplines.\n',
 'natural product reports is a monthly peer-reviewed scientific journal published by the royal society of chemistry. it publishes reviews commissioned by the editorial board on all areas of natural products research. the executive editor is richard kelly.\n',
 'nature chemical biology is a monthly, peer-reviewed, scientific journal, which is published by nature publishing group. it was first published in june 2005 (volume 1, issue 1). terry l. sheppard is a full-time professional editor with the title, "chief editor", and employed by nature chemical biology.[1][2][3]\n',
 'nature chemistry is a monthly peer-reviewed scientific journal published by nature publishing group. it was established in april 2009. the editor-in-chief is stuart cantrill. the journal covers all aspects of chemistry. publishing formats include primary research articles, reviews, news, views, highlights of notable research from other journals, commentaries, book reviews, correspondence. other formats are analysis of issues such as education, funding, policy, intellectual property, and the impact chemistry has on society.\n',
 'nature materials, is a peer-reviewed scientific journal published by nature publishing group. it was launched in september 2002. vincent dusastre is the launching and current chief editor.[1][2][3][4]\n',
 'nature protocols, published by the nature publishing group, is an on-line scientific journal publishing methods in a recipe-style format. the journal was launched in june 2006 and the content includes both classical methods and cutting-edge techniques relevant to the study of biological problems. the content on this site is divided into "nature protocols" and the "protocol exchange".\n',
 'the new journal of chemistry is a monthly peer-reviewed scientific journal publishing research and review articles on all aspects of chemistry. it is published by the royal society of chemistry (rsc) on behalf of the french national centre for scientific research (cnrs). it was established as nouveau journal de chimie in 1977, acquiring its current title in 1999. the editors-in-chief are denise parent (cnrs) and sarah ruthven (rsc). according to the journal citation reports, the journal has a 2019 impact factor of 3.288.[1]\n',
 'open chemistry is a monthly peer-reviewed open access scientific journal covering all fields of chemistry. it is published by walter de gruyter. the editor-in-chief is joaquín plumet, (complutense university).\n',
 'organic & biomolecular chemistry is a weekly peer-reviewed scientific journal covering all aspects of organic chemistry, including organic aspects of chemical biology, medicinal chemistry, natural product chemistry, supramolecular chemistry, macromolecular chemistry, theoretical chemistry, and catalysis. it is published by the royal society of chemistry. its predecessor journals were perkin transactions i and perkin transactions ii. the executive editor is richard kelly.\n',
 'organic letters is a biweekly peer-reviewed scientific journal covering research in organic chemistry. it was established in 1999 and is published by the american chemical society. in 2014, the journal moved to a hybrid open access publishing model. the founding editor-in-chief was amos smith. since 2019, erick m. carreira serves as the editor-in-chief.[1] the journal is abstracted and indexed in: the science citation index expanded, scopus, academic search premier, biosis previews, chemical abstracts service, embase, and medline.[2]\n',
 'organometallics is a biweekly journal published by the american chemical society. its area of focus is organometallic, as well as organometalloid chemistry. this peer-reviewed journal has an impact factor of 3.862 as reported by the 2016 journal citation reports[1] by thomson reuters.\n',
 'perkin transactions is a scientific journal devoted to organic chemistry published from 1997 to 2002 by the royal society of chemistry. it was split into perkin transactions i and perkin transactions ii. the predecessor journals published by the chemical society before the merger of that society with other societies to form the royal society of chemistry were the journal of the chemical society, perkin transactions 1 and journal of the chemical society, perkin transactions 2 (1972-1996). they were replaced by organic and biomolecular chemistry. the name honours the chemist arthur george perkin.\n',
 'photochemical & photobiological sciences is a monthly peer-reviewed scientific journal covering all areas of photochemistry and photobiology. it is published monthly by springer nature and is the official journal of the european photochemistry association, european society for photobiology, asia and oceania society for photobiology, and the korean society of photoscience. the editors-in-chief are dario bassani and santi nonell. according to the journal citation reports, the journal has a 2015 impact factor of 2.235.[1]\n',
 'physical chemistry chemical physics is a weekly peer-reviewed scientific journal publishing research and review articles on any aspect of physical chemistry, chemical physics, and biophysical chemistry. it is published by the royal society of chemistry on behalf of eighteen participating societies. the editor-in-chief is david rueda, (imperial college london, uk).[1]\n',
 'the polish journal of chemistry was a peer-reviewed scientific journal on chemistry and the official journal of the polish chemical society. the journal covered all fields of pure chemistry (physical, theoretical, inorganic, organic and bioorganic) as well as medicinal, macromolecular and supramolecular chemistry, and molecular modelling. it was established in 1921 under the title roczniki chemii (issn\xa00035-7677) and obtained its later title in 1978. the last editor-in-chief was z. galus (university of warsaw).\n',
 'polyhedron is a peer-reviewed scientific journal covering the field of inorganic chemistry. it was established in 1955 as the journal of inorganic and nuclear chemistry and is published by elsevier.\n',
 'the proceedings of the chemical society was a scientific journal published at various times in the life of the chemical society, a scientific society in the united kingdom that combined with other societies to form the royal society of chemistry in 1980. in 1841, the society published memoirs of the chemical society, renamed in 1842 to proceedings of the chemical society. together these were volume 1. volumes 2 and 3 were published as memoirs and proceedings, chemical society, london between 1843 and 1848. the proceedings of the chemical society, london were published from vol. 1, 1885 to vol. 30, 1914 and from 1950 to 1964. between 1915 and 1956 the proceedings of the chemical society, london were published as a supplement to journal of the chemical society, london.\n',
 'rsc advances is an online-only peer-reviewed scientific journal covering research on all aspects of the chemical sciences. it was established in 2011 and is published by the royal society of chemistry. the editor-in-chief is russell j. cox (leibniz university hannover).\n',
 'the revista boliviana de química (issn\xa00250-5460, coden rbqudx) is a bolivian scientific journal in chemistry. it was founded in 1977 and is published by the facultad de ciencias puras y naturales publicación de la carrera de química, carrera de química at the campus universitario cota cota, in la paz. the journal was not published from 1980 to 1982.\n',
 'the revista de la sociedad venezolana química (issn\xa00370-7474, coden rsvqaq), is a venezolanan scientific journal in chemistry. it was founded in 1944 by the sociedad venezolana química (svq), caracas. the latest published volume is 26 (2003).\n',
 'scientia pharmaceutica is a peer-reviewed open access scientific journal publishing original research papers, short communications and a limited number of reviews on all fields of pharmaceutical sciences and related areas, as well as on pharmaceutical practice. since 2008 all articles are published open access.[1]\n',
 '\n',
 'spectroscopy letters is a peer-reviewed scientific journal for rapid communications about work on a wide variety of spectroscopic methods covering all aspects of  spectroscopy.\n',
 'surface science reports is a peer-reviewed scientific journal published by north-holland that covers the physics and chemistry of surfaces. it was established in 1981. it is the review journal corresponding to the journals surface science and surface science letters.\n',
 "synlett is an international scientific journal for accounts and rapid communications of original contributions of fundamental research in synthetic organic chemistry. the impact factor of this journal is 2.419 (2017).[1] nature featured a brief piece by the editor-in-chief of the journal in 2017, benjamin list, where he discussed the journal's experience with the non-traditional peer review system.[2]\n",
 'synthesis is a scientific journal published from 1969 to the present day by thieme. its stated purpose is the "advancement of the science of synthetic chemistry".\n',
 '\ncoordinates: 39°54′35″n 116°19′47″e\ufeff / \ufeff39.9096°n 116.3296°e\ufeff / 39.9096; 116.3296\n',
 'talanta is a peer-reviewed scientific journal in pure and applied analytical chemistry. it was established in 1958 and is published by elsevier, with 15 issues per year. in addition to original research articles, talanta also publishes review articles and short communications.\n',
 'tetrahedron is a weekly peer-reviewed scientific journal covering the field of organic chemistry. according to the journal citation reports, the journal has a 2014 impact factor of 2.641.[1] tetrahedron and elsevier, its publisher, support an annual symposium.[2] in 2010, complaints were raised over its high subscription cost.[3]\n',
 'tetrahedron letters is a weekly international journal for rapid publication of full original research papers in the field of organic chemistry. according to the journal citation reports, the journal has a 2014 impact factor of 2.379 and it is ranked 22nd out of 57 journals in the "organic chemistry" category.[1]\n',
 'theoretical chemistry accounts: theory, computation, and modeling is a peer-reviewed scientific journal publishing original (primary) research and review articles in theoretical chemistry, quantum chemistry, and computational chemistry. it was founded in 1962 as theoretica chimica acta. the publisher is springer berlin heidelberg. the impact factor of this journal is 2.233 (2014).[1] the editor-in-chief is the team of carlo adamo and ilaria ciofini, the associate editor is weitao yang, and the chief advisory editor is donald g. truhlar.\n',
 'trends in analytical chemistry is a peer reviewed journal in analytical chemistry with reviews of the latest developments in the field. its editor as of 2019 is janusz pawliszyn.\n',
 'the zeitschrift für naturforschung (english: journal for nature research) was a peer-reviewed monthly academic journal that was established by kaiser wilhelm institute scientists in 1946. it published original german-language research manuscripts from the fields of chemistry, physics, and the biosciences. it existed for only one year, being split into two parts in 1947, one dealing with physical sciences (zeitschrift für naturforschung a) and the other covering the chemical and biosciences (zeitschrift für naturforschung b). the biosciences were separated in 1973 (zeitschrift für naturforschung c), resulting in three successor journals.[1] the three current journals are all considered to begin with the 1946 volume; consequently, all three journals use the same volume number in each calendar year. the publisher typically uses the abbreviation z. naturforsch. for all three journals with the part indicated in the volume number. for example, an article in a 2012 issue of part a might be referenced in some places as being from volume "67a" of z. naturforsch., but in other places references will show volume "67" of "z. naturforsch. a".\n',
 'the zeitschrift für naturforschung b: a journal of chemical sciences is a monthly peer-reviewed scientific journal. the journal publishes "fundamental studies in all areas of inorganic chemistry, organic chemistry, and analytical chemistry"[1] in both english and german. articles in german are required to be accompanied by an english-language title and abstract.[2] according to the journal citation reports, the journal has a 2014 impact factor of 0.744.[3] the editors-in-chief are gerhard müller (university of konstanz) and annette schier (technical university munich).\n',
 'zeitschrift für physikalische chemie (english: journal of physical chemistry) is a monthly peer-reviewed scientific journal covering physical chemistry that is published by oldenbourg wissenschaftsverlag. its english subtitle is "international journal of research in physical chemistry and chemical physics". it was established in 1887 by wilhelm ostwald, jacobus henricus van \'t hoff, and svante august arrhenius as the first scientific journal for publications specifically in the field of physical chemistry.[1] the editor-in-chief is klaus rademann (humboldt university of berlin).\n' ]

In [48]:
import numpy as np

impact = [22.384,
 None,
 1.104,
 18.808,
 17.083,
 3.864,
 5.977,
 3.286,
 6.785,
 15.335999999999999,
 None,
 None,
None,
 10.745,
 23.643,
 12.703,
 3.5810000000000004,
 2.915,
 1.253,
 1.226,
 2.622,
 4.0969999999999995,
 4.349,
 5.667000000000001,
 1.723,
 3.641,
 2.42,
 4.488,
 1.171,
 5.726,
 1.216,
 3.1639999999999997,
 5.996,
 2.029,
 60.622,
 9.346,
 54.56399999999999,
None,
 1.9409999999999998,
 5.236000000000001,
 1.55,
 9.811,
 2.109,
 3.1239999999999997,
 2.303,
 3.1439999999999997,
 2.753,
 0.5379999999999999,
 2.753,
 3.3819999999999997,
 4.052,
 None,
 38.532,
 3.4210000000000003,
 2.923,
 2.529,
 6.51,
 2.889,
 3.427,
 1.757,
 4.256,
 9.405,
 2.309,
 5.165,
 5.816,
 2.263,
None,
 3.466,
None,
 5.279,
 15.419,
 3.125,
 5.157,
 2.5380000000000003,
 1.444,
 6.921,
 1.758,
 4.5489999999999995,
 3.48,
 1.406,
None,
None,
 2.1959999999999997,
 5.513999999999999,
 1.5,
 4.169,
 1.3019999999999998,
 3.3810000000000002,
 3.3760000000000003,
 3.012,
 2.6,
 0.787,
 2.381,
 6.626,
 7.446000000000001,
 2.011,
 1.403,
 4.257,
 4.805,
 2.369,
 2.6,
 2.991,
 4.189,
 6.475,
 None,
None,
 1.0,
None,
 4.316,
 2.471,
 6.914,
 3.8819999999999997,
None,
 5.914,
None,
 6.232,
None,
None,
 1.704,
 4.4110000000000005,
 11.189,
 11.405999999999999,
 12.587,
 21.686999999999998,
 43.841,
 13.491,
 3.2880000000000003,
 1.216,
 3.8760000000000003,
 6.091,
 3.862,
None,
 2.9019999999999997,
 3.5669999999999997,
None,
 2.108,
None,
 3.36,
None,
None,
None,
 3.3989999999999996,
 0.852,
 9.687999999999999,
 2.369,
None,
None,
 5.3389999999999995,
 2.645,
 2.379,
 2.233,
 9.801,
None,
 1.047,
 1.3559999999999999]

In [52]:
impact

[22.384,
 None,
 1.104,
 18.808,
 17.083,
 3.864,
 5.977,
 3.286,
 6.785,
 15.335999999999999,
 None,
 None,
 None,
 10.745,
 23.643,
 12.703,
 3.5810000000000004,
 2.915,
 1.253,
 1.226,
 2.622,
 4.0969999999999995,
 4.349,
 5.667000000000001,
 1.723,
 3.641,
 2.42,
 4.488,
 1.171,
 5.726,
 1.216,
 3.1639999999999997,
 5.996,
 2.029,
 60.622,
 9.346,
 54.56399999999999,
 None,
 1.9409999999999998,
 5.236000000000001,
 1.55,
 9.811,
 2.109,
 3.1239999999999997,
 2.303,
 3.1439999999999997,
 2.753,
 0.5379999999999999,
 2.753,
 3.3819999999999997,
 4.052,
 None,
 38.532,
 3.4210000000000003,
 2.923,
 2.529,
 6.51,
 2.889,
 3.427,
 1.757,
 4.256,
 9.405,
 2.309,
 5.165,
 5.816,
 2.263,
 None,
 3.466,
 None,
 5.279,
 15.419,
 3.125,
 5.157,
 2.5380000000000003,
 1.444,
 6.921,
 1.758,
 4.5489999999999995,
 3.48,
 1.406,
 None,
 None,
 2.1959999999999997,
 5.513999999999999,
 1.5,
 4.169,
 1.3019999999999998,
 3.3810000000000002,
 3.3760000000000003,
 3.012,
 2.6,
 0.787,
 2.381,
 6.626,
 

In [53]:
df = pd.DataFrame([title, description, impact])

In [55]:
df = df.T

In [57]:
df.columns = ['title', 'description', 'Impact_factor']

In [58]:
df.head()

,title,description,Impact_factor
0,Accounts of Chemical Research,accounts of chemical research is a monthly pee...,22.384
1,Acta Chemica Scandinavica,acta chemica scandinavica was a peer-reviewed ...,None
2,Acta Chimica Slovenica,acta chimica slovenica is a quarterly scientif...,1.104
3,Advanced Functional Materials,advanced functional materials is a peer-review...,18.808
4,Aldrichimica Acta,aldrichimica acta is a scientific journal publ...,17.083


# Application run

In [59]:
key_words = input('please insert the key words of your project, like Analytical Chemistry, Spectroscopy and so on: ')

res = key_words.lower().split()

lst_index = []

for word in res:

    for row in range(df.shape[0]):

        if df['description'].str.contains(word)[row]:
            lst_index.append(row)
            
lst_journals = []
lst_factor = []
dic = {}

for row in lst_index:
    
    journal_name = df['title'][row]
    impact_factor = df['Impact_factor'][row]
    lst_factor.append(impact_factor)
    lst_journals.append(journal_name)
    
    dic = dict(zip(lst_journals, lst_factor))
    
    
df_new = pd.DataFrame(dic.values(), dic.keys()).reset_index()
df_new.columns = ['title', 'Impact_factor']
df_new.sort_values(by = 'Impact_factor', ascending = False)

please insert the key words of your project, like Analytical Chemistry, Spectroscopy and so on: organic


,title,Impact_factor
0,Aldrichimica Acta,17.083
17,Organic Letters,6.091
10,Inorganic Chemistry,5.165
14,The Journal of Organic Chemistry,4.805
6,Bulletin of the Chemical Society of Japan,4.488
16,Organic & Biomolecular Chemistry,3.876
4,Bioorganic & Medicinal Chemistry,3.641
11,Journal of Applied Polymer Science,3.125
9,European Journal of Organic Chemistry,2.889
22,Tetrahedron,2.645
